In [6]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import Linear 
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_undirected, negative_sampling
import networkx as nx
from transformers import AutoModel, AutoTokenizer
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


## Load graph data


In [ ]:

def clean_text_strip(text):
   
    #Método usando strip() y split()
  
    lines = text.strip().split('\r\n')
    return ' '.join(line.strip() for line in lines)

def clean_text_regex(text):

    # Reemplaza cualquier combinación de \r y \n con un espacio
    text = re.sub(r'[\r\n]+', ' ', text)
    # Elimina espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


#texts=load_books_and_split_to_paragraphs("/home/alvaro-montero/Escritorio/Repositorios/GNN_Project/Test text")

libros_ids = [1998, 2680, 1232, 8438, 66048, 3623, 2873, 1004, 1013, 1028, 1059, 1064, 1065,
              1080, 1086, 1119, 1399, 1321, 1301, 1260, 1257, 1228, 1227, 1184, 1245, 3010, 3047,
              3056, 3059, 3152, 3160, 3207, 3289, 3323, 3332, 3435, 3436, 3437, 3438, 3439, 3440,
              3441, 3442, 3443, 3444, 66041, 2500, 2554, 2591, 2600, 2638]  # IDs de los libros seleccionados 3623, 2873, 1998

capitulos = []  # Lista para almacenar capítulos de todos los libros
ids_sin_capitulos = []  # IDs de libros sin capítulos

lista_parrafos = []
titulos_libros = []
diccionario_subjects = []
metadata_grupos_parrafos = []

class TimeoutError(Exception):
    pass


@timeout_decorator.timeout(5, timeout_exception=TimeoutError)
def find_chapters_with_timeout(soup):
    """
    Busca capítulos con un límite de tiempo.
    Lanza TimeoutError si excede el tiempo límite.
    """
    return soup.find_all('div', class_='chapter')

id_parrafo = 0
# Recorrer cada ID de libro
for libro_id in libros_ids:
    url = f'https://www.gutenberg.org/cache/epub/{libro_id}/pg{libro_id}-images.html'

    # Realizar la solicitud HTTP
    response = requests.get(url)

    if response.status_code == 200:
        # Crear un objeto BeautifulSoup para analizar el HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            start_time = time.perf_counter()
            # Encontrar todos los párrafos dentro del capítulo
            parrafos = [clean_text_regex(p.get_text()) for p in soup.find_all('p')]
            end_time = time.perf_counter()
            search_time = end_time - start_time

            # print(f"Búsqueda completada para libro {libro_id} en {search_time:.2f} segundos")
            

            if parrafos and len(parrafos) > 5:
                # URL base de la API de Project Gutenberg
                url = f"https://gutendex.com/books/{libro_id}"  # ID del libro Pride and Prejudice

                # Realizar la solicitud HTTP
                response = requests.get(url)

                # Verificar si la respuesta fue exitosa
                if response.status_code == 200:
                    metadata = response.json()

                    # Obtener el titulo del libro
                    title = metadata.get('title')
                            
                    # Acceder a la sección 'subjects'
                    subjects = metadata.get('subjects', [])
                    diccionario_subjects.append(subjects[0])
                
                    title_tag = title

                    if title_tag:
                        titulo = clean_text_regex(title_tag)
                        titulos_libros.append((libro_id, titulo))
                        print(f"id: {libro_id}, Titulo: {titulo}, tiempo : {search_time:.2f} segundos")
                    else:
                        titulos_libros.append((libro_id, f"Título no encontrado para libro {libro_id}"))
                    
                    # Lista para almacenar todos los capítulos de este libro
                    libro_capitulos = []

                    # Limite inferior para el grupo de parrafos
                    id_parrafo_inf = id_parrafo

                    # libro_capitulos.append(parrafos)
                    # Guardar cada parrafo en la lista general de todos los parrafos
                    for parrafo in parrafos:
                        longitud_parrafo = len(parrafo.split(" "))
                        if longitud_parrafo > 50: # Umbral para la cantidad de palabras por parrafo 
                            lista_parrafos.append(parrafo)
                            id_parrafo += 1
                    # Límite superior para el grupo de parrafos
                    id_parrafo_sup = id_parrafo

                    # Guardamos la información de la lista de parrafos para el libro actual
                    paragraph_data = {
                        "id_libro": libro_id,
                        "id_nodo_inferior": id_parrafo_inf,
                        "id_nodo_superior": id_parrafo_sup,
                        "title": title,
                        "subject": subjects[0]
                    }

                    metadata_grupos_parrafos.append(paragraph_data)

                    id_parrafo += 1
                                    
                    # print("len_lista_parrafos: ", len(lista_parrafos))
                            

                    # Si se encontraron capítulos, almacenarlos con el ID del libro
                    if libro_capitulos:
                        capitulos.append((libro_capitulos))
                else:
                    print("Error al obtener la metadata. Código de estado:", response.status_code)
            else: 
                # Si no se encontraron parrafos, agregar el ID a la lista de libros sin parrafos
                ids_sin_capitulos.append(libro_id)
        except TimeoutError:
            print(f"Timeout al buscar capítulos en libro {libro_id}")
            # Opcional: agregar el libro a una lista de libros que necesitan procesamiento especial
            continue
        except Exception as e:
            print(f"Error al procesar libro {libro_id}: {str(e)}")
            continue
    else:
        print(f"Error al acceder al libro con ID {libro_id}. Código de estado: {response.status_code}")
"""
texts=load_books_and_split_to_paragraphs("/home/alvaro-montero/Escritorio/Repositorios/GNN_Project/Test text")
for text in texts:
    text=eliminar_etiquetas(text)
    text=remover_apostrofes(text)
    text=remover_especiales(text)
    text=dobles_espacios(text)
"""

#texts=[ parrafo for ]
texts=lista_parrafos

mat_len=len(texts)
#adjw_matrix=torch.zeros(mat_len,mat_len)
print(mat_len)

In [ ]:
#Load labels


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")
bert_model = bert_model.to(device)  # Move model to GPU
def get_text_embeddings(texts):
    """Get embeddings for a single batch of texts"""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # Using [CLS] token
    return embeddings

In [ ]:
batch_size=96
dataloader = torch.utils.data.DataLoader(texts, batch_size=batch_size, shuffle=False)
    
embeddings_list = []
with torch.no_grad():
        for batch in tqdm(dataloader, desc="Procesando lotes", unit="lote"):
            batch_embeddings = get_text_embeddings(batch)
            embeddings_list.append(batch_embeddings.cpu())

    # Concatenar todos los embeddings en un solo tensor
node_embeddings = torch.cat(embeddings_list, dim=0)

print("Embeddings de nodos:", node_embeddings.shape)  # Debería ser (num_nodos, 768)
torch.cuda.synchronize() 

In [ ]:
"""
scores=torch.matmul(node_embeddings,node_embeddings.T)    

mask = torch.eye(scores.size(0), device=scores.device).bool()  # Matriz identidad
scores.masked_fill_(mask, float('-inf'))  # Establece -inf en la diagonal
adjw_matrix=F.softmax(scores,dim=1)

#adjw_matrix=torch.triu(adjw_matrix)
print(adjw_matrix)
print(adjw_matrix.shape)
threshold = 1/mat_len
adjw_matrix = torch.where(adjw_matrix < threshold, 
                            torch.zeros_like(adjw_matrix), 
                            adjw_matrix)
    
torch.cuda.synchronize() 
print(adjw_matrix)
edge_index, edge_weight = torch_geometric.utils.dense_to_sparse(adjw_matrix)"""

## Load graph structure

In [3]:
#Load index and weight
index_mat=pd.read_csv("/media/alvaro-montero/9408123808121A36/Ubuntu Programs/Project_gnn/index_matrix.csv",header=None)
weight_mat=pd.read_csv("/media/alvaro-montero/9408123808121A36/Ubuntu Programs/Project_gnn/weight_matrix.csv",header=None)

index_mat=index_mat.to_numpy()
weight_mat=weight_mat.to_numpy()


In [ ]:
def split_graph_data(node_embeddings, adj_matrix, test_size=0.2, random_state=42):
    """
    Split a full graph into train and test sets
    
    Parameters:
    - node_embeddings: tensor of shape [num_nodes, embedding_dim]
    - adj_matrix: adjacency matrix [num_nodes, num_nodes]
    - test_size: proportion of edges to use for testing
    """
    # Get edges from adjacency matrix
    edges = adj_matrix.nonzero()
    edge_index = torch.tensor(np.array(edges), dtype=torch.long)
    
    # Get number of nodes
    num_nodes = adj_matrix.shape[0]
    
    # Split edges into train and test
    # Convert edge_index to list of tuples for splitting
    edge_list = list(zip(edge_index[0].numpy(), edge_index[1].numpy()))
    train_edges, test_edges = train_test_split(edge_list, 
                                             test_size=test_size, 
                                             random_state=random_state)
    
    # Convert back to tensors
    train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
    test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
    
    # Create train adjacency matrix
    train_adj = torch.zeros_like(adj_matrix)
    train_adj[train_edge_index[0], train_edge_index[1]] = 1
    
    # Generate negative edges for testing
    test_neg_edge_index = negative_sampling(
        edge_index=edge_index,
        num_nodes=num_nodes,
        num_neg_samples=test_edge_index.size(1),
        force_undirected=True
    )
    
    return {
        'node_embeddings': node_embeddings,
        'train_edge_index': train_edge_index,
        'test_pos_edge_index': test_edge_index,
        'test_neg_edge_index': test_neg_edge_index,
        'train_adj': train_adj,
        'full_adj': adj_matrix
    }

def create_train_test_masks(edge_index, test_edges, num_nodes):
    """
    Create boolean masks for train and test edges
    """
    # Initialize masks
    train_mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    test_mask = torch.zeros(edge_index.size(1), dtype=torch.bool)
    
    # Convert test edges to set for faster lookup
    test_edges_set = set(map(tuple, test_edges.t().tolist()))
    
    # Create masks
    for idx, (i, j) in enumerate(edge_index.t().tolist()):
        if (i, j) in test_edges_set or (j, i) in test_edges_set:
            train_mask[idx] = False
            test_mask[idx] = True
            
    return train_mask, test_mask

def sample_new_negative_edges(edge_index, num_nodes, num_samples):
    """
    Sample negative edges that don't exist in the graph
    """
    existing_edges = set(map(tuple, edge_index.t().tolist()))
    neg_edges = []
    
    while len(neg_edges) < num_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j and (i, j) not in existing_edges and (j, i) not in existing_edges:
            neg_edges.append((i, j))
            existing_edges.add((i, j))
    
    return torch.tensor(neg_edges, dtype=torch.long).t()

## Modelo


In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch.nn import Linear, ModuleList, Sequential, ReLU, Dropout, Sigmoid

class GATLinkLabelPredictor(torch.nn.Module):
    def __init__(self, 
                 in_channels,     # BERT embedding size (768)
                 hidden_channels=256,
                 label_dim=1,     # Dimension of node labels
                 num_heads=8,
                 num_layers=2,
                 dropout=0.2):
        super(GATLinkLabelPredictor, self).__init__()
        
        self.num_layers = num_layers
        self.dropout = dropout

        # GAT layers for node embedding processing
        self.conv_first = GATConv(
            in_channels + label_dim,  # Concatenate BERT embeddings with labels 
            hidden_channels,
            heads=num_heads,
            dropout=dropout
        )

        self.conv_hidden = ModuleList([
            GATConv(
                hidden_channels * num_heads, 
                hidden_channels,
                heads=num_heads,
                dropout=dropout
            ) for _ in range(num_layers - 1)
        ])

        # Edge predictor
        self.edge_predictor = Sequential(
            Linear(hidden_channels * num_heads * 2, hidden_channels),
            ReLU(),
            Dropout(dropout),
            Linear(hidden_channels, 1),
            Sigmoid()
        )

        # Label predictor for new nodes
        self.label_predictor = Sequential(
            Linear(hidden_channels * num_heads, hidden_channels),
            ReLU(),
            Dropout(dropout),
            Linear(hidden_channels, label_dim),
            Sigmoid()
        )

    def forward(self, x, labels, edge_index):
        # Concatenate node features with labels
        x = torch.cat([x, labels], dim=-1)
        
        # GAT layers
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv_first(x, edge_index)
        x = F.elu(x)

        for conv in self.conv_hidden:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = conv(x, edge_index)
            x = F.elu(x)

        return x

    def predict_links(self, node_embeddings, potential_edges):
        """Predicts probability of links for given node pairs"""
        edge_features_1 = node_embeddings[potential_edges[0]]
        edge_features_2 = node_embeddings[potential_edges[1]]
        edge_features = torch.cat([edge_features_1, edge_features_2], dim=1)
        return self.edge_predictor(edge_features)

    def predict_label(self, node_embeddings):
        """Predicts labels for nodes based on their embeddings"""
        return self.label_predictor(node_embeddings)


/media/alvaro-montero/9408123808121A36/Ubuntu_Programs/Anaconda/envs/gnn_proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATLinkLabelPredictor(
    in_channels=768,    # BERT embedding size
    hidden_channels=256,
    label_dim=3,  # dimension of your labels
    num_heads=8
).to(device)

In [3]:

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parámetros entrenables: {trainable_params}")

Parámetros entrenables: 7360004
